In [37]:
import pandas as pd

year = "2122"

data = pd.read_csv(f"../data/irs/countyoutflow{year}.csv")
county_fips = pd.read_csv("../data/irs/us_county_latlng.csv")

data.columns

Index(['y1_statefips', 'y1_countyfips', 'y2_statefips', 'y2_countyfips',
       'y2_state', 'y2_countyname', 'n1', 'n2', 'agi'],
      dtype='object')

In [38]:
county_fips

,fips_code,name,lng,lat
0,1059,Franklin,-87.843283,34.442381
1,13111,Fannin,-84.319296,34.864126
2,19109,Kossuth,-94.206898,43.204140
3,40115,Ottawa,-94.810589,36.835878
4,42115,Susquehanna,-75.800905,41.821277
...,...,...,...,...
3228,12029,Dixie,-83.158705,29.608068
3229,18017,Cass,-86.346207,40.761660
3230,26091,Lenawee,-84.066412,41.894694
3231,72003,Aguada,-67.175247,18.360392


In [39]:
data = data[
  (data['y1_statefips'] == 6)   & # source state is California
  (data['y1_countyfips'] == 37) & # source county is Los Angeles
  (data['y2_statefips'] <= 56)  & # destination state is real
  (~((data['y2_statefips'] == 6) & (data['y2_countyfips'] == 37))) # destination county is not Los Angeles
]

In [40]:
data['y2_fips'] = data.apply(lambda x: f"{x['y2_statefips']:02d}{x['y2_countyfips']:03d}", axis=1).astype(int)
data

,y1_statefips,y1_countyfips,y2_statefips,y2_countyfips,y2_state,y2_countyname,n1,n2,agi,y2_fips
5638,6,37,6,59,CA,Orange County,21267,34853,2384364,6059
5639,6,37,6,71,CA,San Bernardino County,20446,39796,1292375,6071
5640,6,37,6,65,CA,Riverside County,13038,24863,1070710,6065
5641,6,37,32,3,NV,Clark County,7187,12189,635944,32003
5642,6,37,6,111,CA,Ventura County,6858,11873,1124224,6111
...,...,...,...,...,...,...,...,...,...,...
6179,6,37,48,497,TX,Wise County,20,39,2413,48497
6180,6,37,51,177,VA,Spotsylvania County,20,38,1419,51177
6181,6,37,51,540,VA,Charlottesville city,20,38,1484,51540
6182,6,37,53,75,WA,Whitman County,20,30,2203,53075


In [41]:
merged = pd.merge(data, county_fips, left_on='y2_fips', right_on='fips_code')
merged

,y1_statefips,y1_countyfips,y2_statefips,y2_countyfips,y2_state,y2_countyname,n1,n2,agi,y2_fips,fips_code,name,lng,lat
0,6,37,6,59,CA,Orange County,21267,34853,2384364,6059,6059,Orange,-117.760976,33.703111
1,6,37,6,71,CA,San Bernardino County,20446,39796,1292375,6071,6071,San Bernardino,-116.178378,34.841579
2,6,37,6,65,CA,Riverside County,13038,24863,1070710,6065,6065,Riverside,-115.993902,33.743298
3,6,37,32,3,NV,Clark County,7187,12189,635944,32003,32003,Clark,-115.014609,36.215115
4,6,37,6,111,CA,Ventura County,6858,11873,1124224,6111,6111,Ventura,-119.084846,34.454747
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,6,37,48,497,TX,Wise County,20,39,2413,48497,48497,Wise,-97.654586,33.215766
532,6,37,51,177,VA,Spotsylvania County,20,38,1419,51177,51177,Spotsylvania,-77.656153,38.184535
533,6,37,51,540,VA,Charlottesville city,20,38,1484,51540,51540,Charlottesville,-78.482330,38.037422
534,6,37,53,75,WA,Whitman County,20,30,2203,53075,53075,Whitman,-117.522977,46.901191


In [ ]:
merged.to_csv(f'../data/irs/la_countyoutflow_geocoded_{year}.csv')